In [1]:
from pyscf import gto, scf
import scipy
import inspect
import pyscf.qmmm
import pyscf.dft
import numpy as np
angstrom = 1 / 0.52917721067
from matplotlib import pyplot as plt
from functools import reduce
from numpy.linalg import inv 
from pyscf.scf import cphf 
# ancora il modo migliore per mettere cariche frazionali sulle molecole, se vai a toccare mol.charges si incazza 
#  quando deve calcolare l' initial guess 
def fc(calc,deltaZ):
    mf = pyscf.qmmm.mm_charge(calc, calc.mol.atom_coords()/angstrom, deltaZ)  # now is add_mm_charge
    class NoSelfQMMM(mf.__class__):
        def energy_nuc(self):
            q = self.mol.atom_charges().astype(np.float).copy()
            q1 =q+ np.asarray(deltaZ) 
            return self.mol.energy_nuc(q1)
    return(NoSelfQMMM(mf,mf.mm_mol))

def DeltaV(mol,dL):
    mol.set_rinv_orig_(mol.atom_coords()[0])
    dV=mol.intor('int1e_rinv')*dL[0]
    mol.set_rinv_orig_(mol.atom_coords()[1])
    dV+=mol.intor('int1e_rinv')*dL[1]
    return -dV

In [2]:
co=gto.Mole(atom="H 0 0 0 ;H 0 0 1.4", basis= "sto-3g",units="Bohr",maxit=100)
hfco=scf.RHF(co)
hfco.scf()
dL=.0001

hf1=fc(hfco,[dL,-dL])
hf1.scf()
hf2=fc(hfco,[-dL,dL])
hf2.scf()
dV=DeltaV(co,[dL,-dL])
h2=hfco.get_veff()
dh2=(hf1.get_veff()-hf2.get_veff())/2
J,K=hfco.get_jk()
g_ijkl=hfco.mol.intor('int2e_sph')
g_h2=g_ijkl-np.swapaxes(g_ijkl,1,2)/2

C=hfco.mo_coeff
S=hfco.get_ovlp()
e=hfco.mo_energy
E=np.diag(e)
O=np.diag(hfco.mo_occ)
C1=hf1.mo_coeff
P=hfco.make_rdm1()
P1=hf1.make_rdm1()
P2=hf2.make_rdm1() 
dP=(P1-P2)/2
dC=(abs(C1)*C/abs(C)-C)

Initialize <pyscf.gto.mole.Mole object at 0x7f03f38c79d0> in <pyscf.scf.hf.RHF object at 0x7f03f3eb96d0>
<class 'pyscf.gto.mole.Mole'> does not have attributes  maxit units


converged SCF energy = -0.941480654707798
converged SCF energy = -0.941480680076734
converged SCF energy = -0.941480680076787


In [3]:
U_app= C.T.dot(dV.dot(C))  #+U np.diag(np.diagonal(S)**-0.5) approximate value of 
for i in range(co.nao):
    for j in range(co.nao):
        if e[i]!=e[j]:
            U_app[i][j]/=(e[j]-e[i])
dP_app=C.dot(U_app.dot(O)-O.dot(U_app)).dot(C.T)
print(np.linalg.norm(dP))
print(np.linalg.norm(dP_app))
print(np.linalg.norm(dP_app-dP))
dV_mo=C.T@dV@C
"""
for i in range(10):
    U_app=dV_mo+C.T@np.einsum('ijkl,kl->ij',g_h2,dP_app)@C # approximate value of U
    for i in range(co.nao):
        for j in range(co.nao):
            if e[i]!=e[j]:
                U_app[i][j]/=(e[j]-e[i])
    dP_app=C.dot(U_app.dot(O)-O.dot(U_app)).dot(C.T) """
for k in range(10):
    U_app=dV_mo+C.T@np.einsum('ijkl,kl->ij',g_h2,dP_app)@C # approximate value of U
    for i in range(co.nao):
        for j in range(co.nao):
            if e[i]!=e[j]:
                U_app[i][j]/=(e[j]-e[i])
    dP_app=C.dot(U_app.dot(O)-O.dot(U_app)).dot(C.T)

U=U_app.copy()
print(np.linalg.norm(dP))
print(np.linalg.norm(dP_app))
print(np.linalg.norm(dP_app-dP))
U

0.00035878765618787704
0.00041455727621003555
5.5769620022158494e-05
0.00035878765618787704
0.0003587876610188213
4.8309442745540326e-12


array([[-2.71050543e-20,  1.21177980e-04],
       [-1.21177980e-04, -3.21872520e-20]])

In [4]:
ERI_mo=g_ijkl.copy()
ERI_mo=ERI_mo@C
ERI_mo=(ERI_mo.swapaxes(2,3)@C).swapaxes(3,2)
ERI_mo=(ERI_mo.swapaxes(1,3)@C).swapaxes(3,1)
ERI_mo=(ERI_mo.swapaxes(0,3)@C).swapaxes(3,0)

G_cphf=g_ijkl.copy()
G_cphf=4*G_cphf -G_cphf.swapaxes(3,1)-G_cphf.swapaxes(1,2)
G_cphf=G_cphf@C
G_cphf=(G_cphf.swapaxes(2,3)@C).swapaxes(3,2)
G_cphf=(G_cphf.swapaxes(1,3)@C).swapaxes(3,1)
G_cphf=(G_cphf.swapaxes(0,3)@C).swapaxes(3,0)
"""for i in range(2):
    for a in range(2):
        if e[i] !=e[a] :
            G_cphf[a,i,a,i]/=(e[i]-e[a])""" 
pass

In [5]:
print(np.einsum('ijkl,kl->ij',G_cphf,U@O/2))
print(np.einsum('ijkl,kl->ij',G_cphf[:,:,1:,:1],U[1:,:1]))
print(C.T@np.einsum('ijkl,kl->ij',g_h2,dP)@C)
print(C.T@dh2@C)

[[-6.25708481e-20 -1.19837973e-05]
 [-1.19837973e-05 -9.17701870e-20]]
[[-3.19519636e-20 -1.19837973e-05]
 [-1.19837973e-05 -4.20420574e-20]]
[[ 4.30004746e-17 -1.19837972e-05]
 [-1.19837972e-05  8.73138503e-17]]
[[ 6.42626956e-17 -1.19837972e-05]
 [-1.19837972e-05  3.94073608e-17]]


In [6]:
dV_mo=C.T@dV@C
"""for i in range(co.nao):
    for j in range(co.nao):
        if e[i]!=e[j]:
            dV_mo[i][j]/=(e[j]-e[i])"""
U_app= dV_mo.copy()
for i in range(co.nao):
    for j in range(co.nao):
        if e[i]!=e[j]:
            U_app[i][j]/=(e[j]-e[i])
            
for k in range(10):
   # print( np.einsum('ijkl,kl->ij',G_cphf,U_app@O), C.T@np.einsum('ijkl,kl->ij',g_h2,dP_app)@C)
    U_app=dV_mo+np.einsum('ijkl,kl->ij',G_cphf[:,:,1:,:1],U_app[1:,:1]) # approximate value of U
    for i in range(co.nao):
        for j in range(co.nao):
            if e[i]!=e[j]:
                U_app[i][j]/=(e[j]-e[i])
dP_app=C@(U_app@O-O@U_app)@C.T
U=U_app.copy()
print(np.linalg.norm(dP))
print(np.linalg.norm(dP_app))
print(np.linalg.norm(dP_app-dP))

0.00035878765618787704
0.00035878766101882145
4.830944466214461e-12


In [7]:
A=G_cphf.copy()
for i in range(2):
    for a in range(2):
        if e[i] !=e[a] :
            A[i,a,:,:]/=(e[a]-e[i])

In [8]:
Uprova=np.einsum('ijkl,kl->ij',G_cphf[:,:,1:,:1],U[1:,:1])
for i in range(co.nao):
    for j in range(co.nao):
        if e[i]!=e[j]:
            Uprova[i][j]/=(e[j]-e[i])
print(Uprova)
print(np.einsum('ijkl,kl->ij',A[:,:,1:,:1],U[1:,:1]))


[[-3.19519636e-20 -1.88357910e-05]
 [ 1.88357910e-05 -4.20420574e-20]]
[[-3.19519636e-20 -1.88357910e-05]
 [ 1.88357910e-05 -4.20420574e-20]]


The equation:
$$ V_{mo}+ G_{mo} U = EU-UE 
$$
If $U_0= V_{ai}/(\epsilon_i -\epsilon_a)$
Can be reduced to:
$$ U_0 + (G_{mo}U)_{ai}/(\epsilon_i -\epsilon_a)=U $$
$$(1-A) U =U_0$$
the inverse of $(1-A)$ is the infinite sum: $ \sum_{n=0}^{inf.}A^n $, therefore:
$$ U = \sum_{n=0}^{inf.}A^n U_0 $$

In [9]:
dV_mo=C.T@dV@C
for i in range(co.nao):
    for j in range(co.nao):
        if e[i]!=e[j]:
            dV_mo[i][j]/=(e[j]-e[i])
U_0= dV_mo.copy()

Us=[]
Us.append(U_0)
for k in range(10):
    U=np.sum(u for u in Us)
    dP_app=C@(U@O-O@U)@C.T
    print(np.linalg.norm(dP_app-dP))
    Us.append(np.einsum('ijkl,kl->ij',A[:,:,1:,:1],Us[k][1:,:1])) # approximate value of U
    U=np.sum(u for u in Us)
    dP_app=C@(U@O-O@U)@C.T
    
print(np.linalg.norm(dP_app-dP))
print(Us,U)

5.5769620022158494e-05
8.668772098835158e-06
1.3474708123981396e-06
2.0944454123846985e-07
3.256091328891005e-08
5.056186371823805e-09
7.909801296413789e-10
1.1789791793281737e-10
2.33772287448227e-11
1.4175531956518827e-12
4.830944466214461e-12
[array([[ 1.35525272e-20,  1.40013771e-04],
       [-1.40013771e-04,  1.35525272e-20]]), array([[-3.69185467e-20, -2.17636085e-05],
       [ 2.17636085e-05, -4.85770351e-20]]), array([[ 5.73858408e-21,  3.38291479e-06],
       [-3.38291479e-06,  7.55076853e-21]]), array([[-8.92000099e-22, -5.25837085e-07],
       [ 5.25837085e-07, -1.17368434e-21]]), array([[ 1.38651654e-22,  8.17356207e-08],
       [-8.17356207e-08,  1.82436387e-22]]), array([[-2.15518824e-23, -1.27049078e-08],
       [ 1.27049078e-08, -2.83577400e-23]]), array([[ 3.35000428e-24,  1.97483889e-09],
       [-1.97483889e-09,  4.40790037e-24]]), array([[-5.20721506e-25, -3.06967095e-10],
       [ 3.06967095e-10, -6.85159877e-25]]), array([[ 8.09404600e-26,  4.77146758e-11],
      

/home/giorgiod/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  # This is added back by InteractiveShellApp.init_path()
/home/giorgiod/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  from ipykernel import kernelapp as app


In [10]:
co=gto.Mole(atom="H 0 0 0 ;H 0 0 1.4", basis= "def2-QZVPP",units="Bohr",maxit=100)
hfco=scf.RHF(co)
hfco.scf()
dL=.01

hf1=fc(hfco,[dL,-dL])
hf1.scf()
hf2=fc(hfco,[-dL,dL])
hf2.scf()
dV=DeltaV(co,[dL,-dL])
h2=hfco.get_veff()
dh2=(hf1.get_veff()-hf2.get_veff())/2
J,K=hfco.get_jk()
g_ijkl=hfco.mol.intor('int2e_sph')
g_h2=g_ijkl-np.swapaxes(g_ijkl,1,2)/2

C=hfco.mo_coeff
S=hfco.get_ovlp()
e=hfco.mo_energy
E=np.diag(e)
O=np.diag(hfco.mo_occ)
C1=hf1.mo_coeff
P=hfco.make_rdm1()
P1=hf1.make_rdm1()
P2=hf2.make_rdm1() 
dP=(P1-P2)/2
dC=(abs(C1)*C/abs(C)-C)

Initialize <pyscf.gto.mole.Mole object at 0x7f03f188c450> in <pyscf.scf.hf.RHF object at 0x7f03f188c710>


converged SCF energy = -1.02363963529605
converged SCF energy = -1.02385244745375
converged SCF energy = -1.02385244745983


In [11]:
G_cphf=g_ijkl.copy()
G_cphf=4*G_cphf -G_cphf.swapaxes(3,1)-G_cphf.swapaxes(1,2)
G_cphf=G_cphf@C
G_cphf=(G_cphf.swapaxes(2,3)@C).swapaxes(3,2)
G_cphf=(G_cphf.swapaxes(1,3)@C).swapaxes(3,1)
G_cphf=(G_cphf.swapaxes(0,3)@C).swapaxes(3,0)
A=G_cphf.copy()
for i in range(co.nao):
    for a in range(co.nao):
        if e[i] !=e[a] :
            A[i,a,:,:]/=(e[a]-e[i])

In [12]:
dV_mo=C.T@dV@C
for i in range(co.nao):
    for j in range(co.nao):
        if e[i]!=e[j]:
            dV_mo[i][j]/=(e[j]-e[i])
U_0= dV_mo.copy()
Us=[]
Us.append(U_0)
for k in range(10):
    U=np.sum(u for u in Us)
    dP_app=C@(U@O-O@U)@C.T
    print(np.linalg.norm(dP_app-dP))
    Us.append(np.einsum('ijkl,kl->ij',A[:,:,1:,:1],Us[k][1:,:1])) # insert n_occ
    U=np.sum(u for u in Us)
    dP_app=C@(U@O-O@U)@C.T
    
print(np.linalg.norm(dP))
print(np.linalg.norm(dP_app))    
print(np.linalg.norm(dP_app-dP))


/home/giorgiod/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  # Remove the CWD from sys.path while we load stuff.
/home/giorgiod/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  


0.000917074943842509
0.0003001843627928657
0.00010132662741828594
3.381025522076639e-05
1.125898142503332e-05
4.1026849544015275e-06
2.2869613553437234e-06
2.008390549223877e-06
1.978353451127869e-06
1.975492974516599e-06
0.012374362383678521
0.01237578230448258
1.975262072554131e-06
